In [1]:
import numpy as np            
from matplotlib import pyplot as plt
import matplotlib.animation as animation 
%matplotlib inline
import time, sys
from scipy.sparse import spdiags
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pickle
from scipy.io import savemat,loadmat

## Variables

In [2]:
maxk = 10
convergence_threshold = 1.0e-8

nx = 1001
dx = 2.0 / (nx - 1)
nt = 500 
dt = 0.5 / nt 
c = dt/dx

x=np.linspace(0, 2, nx)

## Functions

In [3]:
def kth_diag_indices(a, k):
    rows, cols = np.diag_indices_from(a)
    if k < 0:
        return rows[-k:], cols[:k]
    elif k > 0:
        return rows[:-k], cols[k:]
    else:
        return rows, cols

In [4]:
def sine_wave(amp,width):
    
    u0 = np.zeros(nx) + 1.0
    u0[1:int(width/dx+1)] = 1.0 + amp/2*(np.sin(2*np.pi/(x[int(width/dx+1)]-x[1])*x[1:int(width/dx+1)]-np.pi/2)+1)
    u0[-1] = u0[0]
    
    return u0

In [5]:
def residual(un,uw,c,idxn1):
    
    # r = -u^{n} + u^{n+1} -dt*f(u^{n+1})
    
    f = c*(uw**2 - uw*uw[idxn1]) 
    
    r = -un + uw + f
    
    return r

In [6]:
def jacobian(u,c,idxn1):

    # J = I - dt*dfdu
    
    diag_comp = 1.0 + c*(2*u - u[idxn1])
    subdiag_comp = np.ones(nx-1)
    subdiag_comp[:-1] = -c*u[1:]
        
    data = np.array([diag_comp, subdiag_comp])
    J = spdiags(data,[0,-1],nx-1,nx-1,format='csr')
    J[0,-1] = -c*u[0]
    
    return J

#     diag_comp = 1.0 + c*(2*u - u[idxn1])
#     subdiag_comp = -c*u
    
#     J = np.diag(diag_comp,0)
#     J[dn1] = subdiag_comp[1:]
#     J[0,-1] = -c*u[0]
    
#     return J

In [7]:
def solve(u0):

    u = np.zeros((nt+1,nx))
    u[0] = u0
    I = sparse.eye(nx,format='csr')
    for n in range(nt): 
        uw = u[n,:-1].copy()
        r = residual(u[n,:-1],uw,c,idxn1)
        
        for k in range(maxk):
            J = jacobian(uw,c,idxn1)
            duw = spsolve(J, -r)
#             duw = np.linalg.solve(J,-r)
            uw = uw + duw
            r = residual(u[n,:-1],uw,c,idxn1)

            rel_residual = np.linalg.norm(r)/np.linalg.norm(u[n,:-1])
            if rel_residual < convergence_threshold:
                u[n+1,:-1] = uw.copy()
                u[n+1,-1] = u[n+1,0]
                break
    
    return u

## Build FOM solutions

In [8]:
dn1=kth_diag_indices(np.eye(nx-1),-1)
idxn1=np.zeros(nx-1,dtype='int')
idxn1[1:]=np.arange(nx-2)
idxn1[0]=nx-2

In [11]:
amp_arr = np.array([0.6,0.62,0.64,0.66,0.68,0.70,0.72,0.74,0.76,0.78,0.8,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96,0.98,1.0,\
                    1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16,1.18,1.2])
num_amp=amp_arr.shape[0]
for i in range(num_amp):
    amp=amp_arr[i]
    print(amp)

0.8
0.82
0.84
0.86
0.88
0.9
0.92
0.94
0.96
0.98
1.0
1.02
1.04
1.06
1.08
1.1
1.12
1.14
1.16
1.18
1.2


In [12]:
ampArr = np.array([0.6,0.62,0.64,0.66,0.68,0.70,0.72,0.74,0.76,0.78,0.8,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96,0.98,1.0,\
                    1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16,1.18,1.2])
num_amp=ampArr.shape[0]
for i in range(num_amp):
    amp=ampArr[i]
    print(amp)
    u0=sine_wave(amp,1.0)
    u=solve(u0)
    file_name_FOM="./data/FOM_amp_{}.p".format(amp)
    pickle.dump(u, open(file_name_FOM, "wb"))

0.6
0.62
0.64
0.66
0.68
0.7
0.72
0.74
0.76
0.78
